In [2]:
from collections import defaultdict

def euler_directed(nodes, edges):
    # build adjacency with edge ids, and in/out-degree
    out_adj = defaultdict(list)
    indeg = defaultdict(int)
    outdeg = defaultdict(int)
    for i, (u, v) in enumerate(edges):
        out_adj[u].append((v, i))
        outdeg[u] += 1
        indeg[v] += 1

    # active nodes = vertices that appear in at least one edge
    active = {u for u, _ in edges} | {v for _, v in edges}
    if not active:
        return None  # no edges → keep it simple: treat as not found

    # degree conditions:
    #   Euler circuit: indeg == outdeg for all
    #   Euler trail  : exactly one (out-in)=+1 (start) and one (out-in)=-1 (end); others 0
    start = None
    plus1 = minus1 = 0
    for v in active:
        diff = outdeg[v] - indeg[v]
        if diff == 1:
            plus1 += 1; start = v
        elif diff == -1:
            minus1 += 1
        elif diff == 0:
            pass
        else:
            return None
    if (plus1, minus1) not in [(0, 0), (1, 1)]:
        return None

    # choose start: the +1 node if trail, otherwise any node with outdegree>0
    if start is None:
        start = next((v for v in active if outdeg[v] > 0), None)
        if start is None:
            return None

    # weak connectivity check on underlying undirected graph
    undirected = defaultdict(list)
    for u, v in edges:
        undirected[u].append(v)
        undirected[v].append(u)
    seen, stack = set(), [start]
    while stack:
        u = stack.pop()
        if u in seen: 
            continue
        seen.add(u)
        for w in undirected[u]:
            if w not in seen:
                stack.append(w)
    if seen & active != active:
        return None

    # Hierholzer
    used = [0] * len(edges)
    path = []
    stack = [start]
    while stack:
        u = stack[-1]
        # drop already used edges at the tail
        while out_adj[u] and used[out_adj[u][-1][1]]:
            out_adj[u].pop()
        if not out_adj[u]:
            path.append(stack.pop())
        else:
            v, eid = out_adj[u].pop()
            if not used[eid]:
                used[eid] = 1
                stack.append(v)

    # must use all edges
    if sum(used) != len(edges):
        return None
    return path[::-1]


In [3]:
def run_program_directed(nodes, edges):
    res = euler_directed(nodes, edges)
    if res is None:
        print("euler path not found")
    else:
        print(" -> ".join(map(str, res)))


In [4]:
print("directed: Eulerian circuit")
nodes = [1,2,3]
edges = [(1,2),(2,3),(3,1)]
run_program_directed(nodes, edges)

print("\ndirected: Eulerian path (trail, not circuit)")
nodes2 = ['a','b','c','d']
edges2 = [('a','b'),('b','c'),('c','a'),('a','d')]  # start 'a', end 'd'
run_program_directed(nodes2, edges2)

print("\ndirected: not Eulerian (imbalanced)")
nodes3 = [1,2]
edges3 = [(1,2),(1,2)]
run_program_directed(nodes3, edges3)


directed: Eulerian circuit
1 -> 2 -> 3 -> 1

directed: Eulerian path (trail, not circuit)
a -> b -> c -> a -> d

directed: not Eulerian (imbalanced)
euler path not found
